# Introduction

Most frequently visited places in Toronto. 

Canada is a beautiful and diverse country. It is a great place to visit for people from any country in the world. Given the vast variety of ethnicity, language, culture, and food, Canada is one of the most amicable places to visit or live. In this project, we will explore popular venues in Toronto.

# Data

Data: We will take the Canada postal code data from Wikipedia. 
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

# Methodology

We will use BeautifulSoup to scrape the Wikipedia data of Canada postal code information. We will group the Canada neighborhood and Borough with postal code using #Python #Dataframe codes. Using #Geocoder, we will assign the latitude and longitude, and find the #geographical coordinates of Toronto, Ontario. Then we will install #folium for visualization of Toronto location on map. Then we will obtain most popular neighborhoods in Toronto. Further, we will use #Foursquare to obtain top 100 venues within 500 meters range of popular neighborhood https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}. We will extract venue groups such as food, gym, sports, parks, airports, and video games etc., and obtain the top most common venues categories for each neighborhood. Then we will cluster the venues of each neighborhood and finally we will run a test to verify venues in each cluster.

In [1]:
#Firstly import the required data analysis programs

import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
#Scrap the wikipedia data containing the postal codes of Canada to obtain sytematically arranged information of postal codes with respective Borough and Neighbourhood


source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head()

df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head()

df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df.head()

df["Borough"] = df["Borough"].str.replace("\n","")
df.head()

df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [3]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [4]:
#Merge both information into a single enumeration

df_toronto = pd.merge(df, df_geo_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A,Not assigned,Not assigned,NaN,NaN
1,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
4,M1G,Scarborough,Woburn,43.770992,-79.216917


In [5]:
#Now, obtain the latitude and longitude of Toronto


address = "Toronto, ON"
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [6]:
#Install folium to create Toronto geographical map

!pip install folium==0.5.0

     |████████████████████████████████| 79 kB 5.0 MB/s eta 0:00:011
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=9134e7775d68a0e3a82dbab42d45753c8f4dba9b1d92e3a690f0345cfddca13c
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium


In [9]:
import folium

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [8]:
#Add marker for Toronto on the map


address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

import folium

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [43.6534817, -79.3839347],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [11]:
#Obtain the popular neighbourhood location data in Toronto


df_toronto_denc = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_denc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [12]:
#Mark neigbourhoods of Toronto on map


map_toronto_denc = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_toronto_denc['Latitude'], 
        df_toronto_denc['Longitude'], 
        df_toronto_denc['Borough'], 
        df_toronto_denc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_denc)  

map_toronto_denc

In [13]:
#Enter Foursquare cerdentials to use foursquare for obtaining venues name and details


CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = ''

##Foursquare account details have been removed for publishing purpose

In [14]:
#Explore the first neighbourhood


neighborhood_name = df_toronto_denc.loc[0, 'Neighborhood']
print(f"The first neighborhood's name is '{neighborhood_name}'.")

The first neighborhood's name is 'The Beaches'.


In [15]:
#Obtain the location details


neighborhood_latitude = df_toronto_denc.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto_denc.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [16]:
#Gather the list of top 100 venues within 500 meters range of the beach neighbourhood


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [17]:
#Extract venue categories


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
#Clean the data and obtain in dataframe format


venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
#Extract venues categories for each neighbourhood


toronto_denc_venues = getNearbyVenues(names=df_toronto_denc['Neighborhood'],
                                   latitudes=df_toronto_denc['Latitude'],
                                   longitudes=df_toronto_denc['Longitude']
                                  )

In [22]:
#Check no. of venues obtained for each neighbourhood


toronto_denc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,61,61,61,61,61,61
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,65,65,65,65,65,65
Christie,16,16,16,16,16,16
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [24]:
#Find out unique categories for each neighbourhood


print('There are {} uniques categories.'.format(len(toronto_denc_venues['Venue Category'].unique())))

There are 232 uniques categories.


In [25]:
#Analyze each neighbourhood individually


# one hot encoding
toronto_denc_onehot = pd.get_dummies(toronto_denc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_denc_onehot['Neighborhood'] = toronto_denc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_denc_onehot.columns[-1]] + list(toronto_denc_onehot.columns[:-1])
toronto_denc_onehot = toronto_denc_onehot[fixed_columns]

toronto_denc_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
#Group the data by neighbourhood and obtain the mean values


toronto_denc_grouped = toronto_denc_onehot.groupby('Neighborhood').mean().reset_index()
toronto_denc_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.016393,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.062500,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
4,Central Bay Street,0.015385,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.015385,0.0,0.0,0.015385


In [27]:
#Obtain the top twenty most common venue in all the neighbourhoods


def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_denc_grouped['Neighborhood']

for ind in np.arange(toronto_denc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_denc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Beer Bar,Restaurant,Pharmacy,Farmers Market,Cheese Shop,...,Creperie,Concert Hall,Comfort Food Restaurant,Greek Restaurant,Nightclub,Lounge,Café,Breakfast Spot,Park,Liquor Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bakery,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant,...,Italian Restaurant,Intersection,Bar,Stadium,Nightclub,Gym,Grocery Store,Furniture / Home Store,Fountain,Cuban Restaurant
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,...,Butcher,Restaurant,Burrito Place,Brewery,Skate Park,Park,Athletics & Sports,Diner,Ethiopian Restaurant,Escape Room
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,...,Airport Gate,Airport Food Court,Airport,Bar,Escape Room,Diner,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bank,Bubble Tea Shop,Burger Joint,Thai Restaurant,...,Japanese Restaurant,Juice Bar,Korean Restaurant,Middle Eastern Restaurant,Pizza Place,Office,Park,Ice Cream Shop,Poke Place,Portuguese Restaurant


In [28]:
#Cluster the neighbourhood


# set number of clusters
kclusters = 10

toronto_denc_grouped_clustering = toronto_denc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_denc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 9, 0, 7, 0, 0, 0, 8], dtype=int32)

In [29]:
#Include top twenty venues in cluster


# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_denc_merged = df_toronto_denc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_denc_merged = toronto_denc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_denc_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,5,Trail,Health Food Store,Pub,Wine Bar,...,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Cuban Restaurant,Cupcake Shop,Ethiopian Restaurant,Creperie
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,...,Japanese Restaurant,Indian Restaurant,Grocery Store,Frozen Yogurt Shop,Fruit & Vegetable Store,Spa,Dessert Shop,Cosmetics Shop,Caribbean Restaurant,Café
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Fast Food Restaurant,Sushi Restaurant,Pet Store,Fish & Chips Shop,...,Italian Restaurant,Gym,Park,Brewery,Board Shop,Movie Theater,Sandwich Place,Steakhouse,Dessert Shop,Department Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Bakery,Gastropub,Brewery,...,Pizza Place,Pet Store,Park,Yoga Studio,Bar,Coworking Space,Middle Eastern Restaurant,Diner,Latin American Restaurant,Italian Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Bus Line,Swim School,Park,College Gym,...,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,College Rec Center,Escape Room,Cupcake Shop


In [30]:
#Visualize the cluster on map


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_denc_merged['Latitude'], 
        toronto_denc_merged['Longitude'], 
        toronto_denc_merged['Neighborhood'], 
        toronto_denc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
#Test cluster by using the cluster label information to verify the venue details #Cluster 1


toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 0, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,East Toronto,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Bookstore,Ice Cream Shop,Pub,Pizza Place,...,Japanese Restaurant,Indian Restaurant,Grocery Store,Frozen Yogurt Shop,Fruit & Vegetable Store,Spa,Dessert Shop,Cosmetics Shop,Caribbean Restaurant,Café
2,East Toronto,0,Fast Food Restaurant,Sushi Restaurant,Pet Store,Fish & Chips Shop,Ice Cream Shop,Burrito Place,Restaurant,Liquor Store,...,Italian Restaurant,Gym,Park,Brewery,Board Shop,Movie Theater,Sandwich Place,Steakhouse,Dessert Shop,Department Store
3,East Toronto,0,Coffee Shop,Bakery,Gastropub,Brewery,Café,American Restaurant,Comfort Food Restaurant,Seafood Restaurant,...,Pizza Place,Pet Store,Park,Yoga Studio,Bar,Coworking Space,Middle Eastern Restaurant,Diner,Latin American Restaurant,Italian Restaurant
6,Central Toronto,0,Clothing Store,Coffee Shop,Seafood Restaurant,Salon / Barbershop,Restaurant,Café,Chinese Restaurant,Park,...,Mexican Restaurant,Diner,Italian Restaurant,Fast Food Restaurant,Gym / Fitness Center,Gift Shop,Spa,Yoga Studio,BBQ Joint,Dessert Shop
7,Central Toronto,0,Dessert Shop,Sandwich Place,Pizza Place,Italian Restaurant,Café,Coffee Shop,Sushi Restaurant,Gym,...,Seafood Restaurant,Indian Restaurant,Park,Pharmacy,Farmers Market,Restaurant,Brewery,Gourmet Shop,Greek Restaurant,Gas Station
9,Central Toronto,0,Coffee Shop,Sushi Restaurant,American Restaurant,Supermarket,Pub,Bank,Fried Chicken Joint,Pizza Place,...,Liquor Store,Light Rail Station,Vietnamese Restaurant,Sandwich Place,Deli / Bodega,Distribution Center,Diner,Dessert Shop,Department Store,Dog Run
11,Downtown Toronto,0,Coffee Shop,Café,Park,Pizza Place,Chinese Restaurant,Market,Pub,Bakery,...,Restaurant,Pet Store,Sandwich Place,Deli / Bodega,Butcher,Farmers Market,Indian Restaurant,Plaza,Beer Store,Playground
12,Downtown Toronto,0,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Gay Bar,Yoga Studio,Hotel,Mediterranean Restaurant,...,Café,Fast Food Restaurant,Pub,Park,Korean Restaurant,Caribbean Restaurant,Clothing Store,Burrito Place,Pizza Place,Mexican Restaurant
13,Downtown Toronto,0,Coffee Shop,Pub,Park,Café,Bakery,Theater,Breakfast Spot,Brewery,...,Farmers Market,Shoe Store,Electronics Store,Yoga Studio,Restaurant,Distribution Center,Dessert Shop,Chocolate Shop,Performing Arts Venue,Mexican Restaurant
14,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Lingerie Store,Japanese Restaurant,...,Theater,Diner,Pizza Place,Bookstore,Fast Food Restaurant,Italian Restaurant,Movie Theater,Modern European Restaurant,Burrito Place,New American Restaurant


In [33]:
#Cluster 2 test


toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 1, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
23,Central Toronto,1,Park,Jewelry Store,Trail,Sushi Restaurant,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,...,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant,Escape Room,Creperie,Coworking Space


In [34]:
#Cluster 3 test


toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 2, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
22,Central Toronto,2,Garden,Wine Bar,Dance Studio,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,...,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Cupcake Shop,Event Space,Cuban Restaurant,Creperie,Coworking Space


In [35]:
#Cluster 4 test


toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 3, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
10,Downtown Toronto,3,Park,Trail,Playground,Cuban Restaurant,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,...,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cupcake Shop,Creperie,Escape Room,Coworking Space,Cosmetics Shop


In [36]:
#Cluster 5 test


toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 4, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
4,Central Toronto,4,Bus Line,Swim School,Park,College Gym,College Cafeteria,Electronics Store,Eastern European Restaurant,Donut Shop,...,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,College Rec Center,Escape Room,Cupcake Shop


# Result
Here by using various data analysis tools such as Python dataframes, Folium, and Foursquare, we obtained:
1) Location data of Canada and marked that on map.
2) Location data of Toronto and marked that on map.
3) Neighborhood details of Toronto.
4) Top 100 Venues within 500 meters range of each neighborhood.
5) Twenty common venue categories of each neighborhood.
6) Clustered and verified information of venues.

# Discussion
By employing data analysis tool, we have obtained popular Toronto neighborhood location data. In addition, we found the most common categories of venues in each neighborhood cluster. For example, in Berczy Park neighborhood, coffee shop is the most common category visited. Likewise, Cafe is the most common category in the neighborhood cluster involving Brockton, Parkdale Village, and Exhibition Place. Further, if someone is looking for Italian restaurant, they must go to the Central Bay Street neighborhood in Toronto. Thus, by using data science tools, we can easily convert the Wikipedia data information to a much more organized and presentable form. Further, by using various location program such as Geocoder, data visualization program such as Folium, and venue finding application such as Foursquare, we can easily locate a place on map, and find the neighboring avenues or points of interest.

# Conclusion
Data science utilizes various data analysis tools to make the data presentable and discernible that can also provide focused information to the stakeholders to meet the deliverables. In this capstone project, we saw how the data science is used to gain information about a particular neighborhood. We, by using BeautifulSoup, Python dataframe, Folium, and Foursquare, have identified the most commonly visited places in various neighborhoods in Toronto.